# Thông tin nhóm 14

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Tình hình tội phạm.
- ...

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# Lấy dữ liệu - parse HTML

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại  California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

In [1]:
# Import các thư viện cần thiết.
import urllib.robotparser
import json
import os.path
import re

from requests_html import HTMLSession

In [ ]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()

In [ ]:
# Lấy link chứa danh sách các ngôi nhà đã bán theo Quận tại California
check_parse = rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

base_url = "https://m.realtytrac.com"
list_url_county = []

# Kiểm tra việc parse HTML được cho phép hay không?
if check_parse == True:
    session = HTMLSession()
    r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

    counties = r.html.find("option")

    for county in counties:
        tail_url = county.attrs['value']
        if (tail_url):
            url = base_url + tail_url
            list_url_county.append(url)


list_url_county

In [ ]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt.
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1


# Hàm parseDetailPage: parse để lấy thông tin chi tiết của từng căn nhà từ detailUrl (của căn nhà đã bán).
# Tham số: session và detailUrl của căn nhà cần parse.
# Trả về: dictionary chứa các thông tin đã parse được.
def parseDetailPage(session: HTMLSession, detailUrl: str):

    # Lấy mã HTML của trang web.
    r = session.get(detailUrl)

    # Dictionary lưu lại kết quả.
    result = {}

    # Lấy address của estate.
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text

    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text

    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text

    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text

    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code

    result["address"] = address

    # Lấy thông tin ngày đã bán căn nhà.
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None

    # Lấy giá cho thuê trước khi bán.
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None


    # Lấy phân mô tả chi tiết về căn nhà.
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)  # sở dĩ lấy mà HTML để dễ parse sau này (khi muốn lấy thông tin từ details)
    result["details"] = details

    # Lấy property info của căn nhà.
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    # Lấy property taxes của căn nhà.
    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes

    # Lấy số lượng trường học gần đó.
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))

    result["school"] = num_school

    # Lấy thông tin tội phạm trong vùng.
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value

    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó.
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc

    return result

In [ ]:
# Hàm parseSearchPage: parse theo page (mỗi parse chứa danh sách và url của 25 căn nhà đã được bán).
# Tham số: session và url của page hiện tại.
# Trả về: list thông tin chi tiết của tất cả căn nhà tại page hiện tại và url của page kế tiếp.
def parseSearchPage(session: HTMLSession, urlPage: str):

    # Lấy mã HTML.
    r = session.get(urlPage)

    # Lấu detailUrl của tất cả bất động sản.
    detailUrls = [link for link in r.html.absolute_links if 'property' in link]

    results = []  # List chứa kết quả parse được tại page.

    # Tiến hành parse từ detailUrl
    for detailUrl in detailUrls:
        # Kiểm tra việc parse HTML được cho phép hay không?
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue

        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)

    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [ ]:
# Hàm getAllCountyProperties: parse tất cả các page của một County để lấy thông tin chi tiết của 
# căn nhà được bán thành công.
# Tham số: session và countryUrl chứa thông tin các căn nhà được bán của một County.
# Trả về: File chứa kết quả đã parse thành công (tên file là tên County được parse).
def getAllCountyProperties(session: HTMLSession, countyUrl: str):
    # Lấy tên County từ urlCounty.
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"

    # Kiểm tra sự tồn tại file.
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return

    # Mở file để ghi dữ liệu (kiểu file là json)
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 1):
            # Kiểm tra việc parse HTML được cho phép hay không?
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue

            # Tiến hành parse theo từng page.
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)

            # Ghi kết quả xuống file.
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")

            # Kiểm tra có còn page tiếp theo hay không?
            if not nextPage:
                break

            # Mỗi lần parse 1 page thì cho sleep 2s.
            time.sleep(2)

Tiến hành lấy dữ liệu. Lưu ý trong quá trình lấy có thể bị timeout do đường truyền mạng, nếu bị thì chỉ cần làm theo các bước sau:
- Xem đã lấy tới page mấy. vd: "...page-125"
- Đổi lại tên file trong thư mục data là "orange-county-1.json" chẳng hạn (số 1, 2, 3,.. là theo mỗi lần lấy dữ liệu của mình).
- Vào hàm "getAllCountyProperties" đổi lại dòng "for url in urlGenerator(countyUrl, 1)" thành "for url in urlGenerator(countyUrl, 125)", số 125 là tùy mình lấy tới page thứ mấy mà bị timeout.
- Dữ liệu tất nhiên sẽ bi trùng, cứ việc yên tâm vì phần sau sẽ xóa trung.

In [ ]:
url = "https://m.realtytrac.com/mapsearch/sold/ca/orange-county"
session = HTMLSession()
getAllCountyProperties(session, url)

In [ ]:
# Phần gộp từng phần dữ liệu đã lấy dang dở lại thành 1 file.
# Nếu không bị timeout như trường hợp ở trên thì không dùng shell code này.
inputFiles = ["data/orange-county-1.json", "data/orange-county-2.json", 
              "data/orange-county-3.json", "data/orange-county-4.json"]
outputFile = "data/full-orange-county.json"

with open(outputFile, "a+") as fOut:
    for inputFile in inputFiles:
        with open(inputFile, "r") as fIn:
            data = fIn.read()
            fOut.write(data)

In [ ]:
# Phần convert file json sang file csv.

def xstr(s):
    return '' if s is None else str(s)


# Hàm preprocessFile: chuyển file json sang file csv.
# Tham số: tên file json và tên file csv.
# Không trả về.
def preprocessFile(inputFile: str, outputFile: str):

    # Kiểm tra tồn tại file Input.
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return

    # Kiểm tra tồn tại file Output.
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return

    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )

        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t".
        fOut.write(f"{tab.join(headers)}\n")

        # Duyệt từng line trong file Input (json).
        for line in fIn:
            row = json.loads(line)

            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")

            # Lấy ngày bán thành công.
            date_sold = row.get("date_sold")

            # Lấy giá cho thuê mỗi tháng (trước khi được bán).
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))

            # Lấy phần info
            info = row.get("info")

            # Type.
            info_type = info.get("type")

            # Bedrooms.
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)

            # Bathrooms.
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)

            # Size.
            info_size = info.get("size")
            if info_size == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",", "").replace(" sqft", ""))

            # Lot size.
            info_lot_size = info.get("lot_size")
            if info_lot_size == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",", "").replace(" sqft", "").replace(" acres", ""))

            # Year build.
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)

            # Est value.
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$", "").replace(",",""))

            # Sold price.
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$", "").replace(",", ""))

            # ID.
            info_property_id = info.get("property_id")

            # County.
            info_county = info.get("county")

            # Parcel_number.
            info_parcel_number = info.get("parcel_number")

            # Lấy phần Taxes.
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$", "").replace(",", ""))
                taxes_improvements = float(taxes.get("improvements").replace("$", "").replace(",", ""))
                taxes_total = float(taxes.get("total").replace("$", "").replace(",", ""))
                taxes_taxes = taxes.get("taxes").replace("$", "").replace(",", "")

            # Lấy số lượng scholl gần đó.
            school = int(row.get("school"))

            # Lấy tình hình tội phạm.
            crime = row.get("local_crime_index")
            if crime is not None:
                total_crime = crime.get("total_crime")
                if total_crime is not None:
                    total_crime = int(total_crime)
                violent_crime = crime.get("violent_crime")
                if violent_crime is not None:
                    violent_crime = int(violent_crime)
                property_crime = crime.get("property_crime")
                if property_crime is not None:
                    property_crime = int(property_crime)

            # Lấy số lượng foreclosures gần đó.
            foreclosures = int(row.get("foreclosures"))

            # Row này là tổng hợp các thông tin lấy được ở trên.
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )

            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

In [ ]:
inputFile = "data/full-orange-county.json"
outputFile = "data/orange-county.csv"
preprocessFile(inputFile, outputFile)

# Tiền xử lý dữ liệu

In [2]:
# Import các thư viện cần thiết.
import pandas as pd

## Xóa dữ liệu trùng

### Dữ liệu ban đầu

In [3]:
file_csv = "data/Orange-County/orange-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21268 entries, 0 to 21267
Data columns (total 26 columns):
address_street        21204 non-null object
address_locality      21266 non-null object
address_region        21268 non-null object
address_code          21268 non-null int64
date_sold             21268 non-null object
mortgage              21134 non-null float64
info_type             21268 non-null object
info_bedrooms         19148 non-null float64
info_bathrooms        19159 non-null float64
info_size             20282 non-null float64
info_lot_size         14829 non-null float64
info_year_built       19737 non-null float64
info_est_value        1297 non-null float64
info_sold_price       19837 non-null float64
info_property_id      21268 non-null int64
info_county           21268 non-null object
info_parcel_number    21268 non-null int64
taxes_land            20981 non-null float64
taxes_improvements    20981 non-null float64
taxes_total           20981 non-null float64
taxe

Dữ liệu ban đầu có 21268 dòng.

### Dữ liệu sau khi xóa trùng

In [5]:
unique_data_df = data_df.drop_duplicates()

In [6]:
unique_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18893 entries, 0 to 21267
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

Dữ liệu sau khi xóa trùng còn lại 18893 dòng.

In [7]:
# Ghi dữ liệu xuống file csv.
unique_data_df.to_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t', index=False)

### Lấy dữ liệu của năm 2019

In [8]:
df = pd.read_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t')
df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18893 entries, 0 to 18892
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

In [10]:
df = df.assign(year_sold=df.date_sold.str[-4:])

In [11]:
df = df[df.year_sold == "2019"]
df.year_sold = df.year_sold.astype(int)

In [12]:
df.year_sold.unique()

array([2019])

In [13]:
# Ghi dữ liệu xuống file csv.
df.to_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t', index=False)

In [14]:
data_df = pd.read_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t')

In [15]:
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4,2019
1,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4,2019
2,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4,2019
3,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4,2019
4,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4,2019
5,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4,2019
6,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4,2019
7,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4,2019
8,13887 La Jolla Plz,Garden Grove,CA,92844,11/22/2019,2226.0,Single Family Residence,3.0,3.0,1637.0,...,332109.0,115034.0,447143.0,5673 (1.26 %),3,76.0,57.0,122.0,4,2019
9,163 Jaripol Cir,Rancho Mission Viejo,CA,92694,11/22/2019,1881.0,Condominium,1.0,2.0,921.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,2019


In [17]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16011 entries, 0 to 16010
Data columns (total 27 columns):
address_street        15963 non-null object
address_locality      16009 non-null object
address_region        16011 non-null object
address_code          16011 non-null int64
date_sold             16011 non-null object
mortgage              15976 non-null float64
info_type             16011 non-null object
info_bedrooms         14434 non-null float64
info_bathrooms        14444 non-null float64
info_size             15204 non-null float64
info_lot_size         10814 non-null float64
info_year_built       14734 non-null float64
info_est_value        221 non-null float64
info_sold_price       15755 non-null float64
info_property_id      16011 non-null int64
info_county           16011 non-null object
info_parcel_number    16011 non-null int64
taxes_land            15753 non-null float64
taxes_improvements    15753 non-null float64
taxes_total           15753 non-null float64
taxes

Dữ liệu có 16011 căn nhà được bán trong 2019

### Xem phần street

In [21]:
tmp = data_df.address_street.str.replace(r'[^a-zA-Z ]', '').str.strip()

In [22]:
tmp

0              Silver Crk
1                   Alava
2               Hansen Ln
3                 Copante
4              Burwood St
               ...       
16006    N Torrey Pine Ln
16007        Kingswood Ln
16008         Tomahawk St
16009          Faeroe Bay
16010          Garner Way
Name: address_street, Length: 16011, dtype: object

In [23]:
len(tmp.unique())

9765

### Xem phần locality

In [46]:
list_locality = data_df.address_locality.unique()
print(list_locality)
print(len(list_locality))

['Irvine' 'Mission Viejo' 'Huntington Beach' 'La Habra' 'Santa Ana'
 'Buena Park' 'Garden Grove' 'Rancho Mission Viejo'
 'Rancho Santa Margarita' 'Fountain Valley' 'Newport Beach' 'Aliso Viejo'
 'Fullerton' 'Westminster' 'Anaheim' 'Yorba Linda' 'Silverado' 'Tustin'
 'Placentia' 'Newport Coast' 'Costa Mesa' 'San Clemente' 'Los Alamitos'
 'Laguna Niguel' 'Cypress' 'Seal Beach' 'San Juan Capistrano' 'Orange'
 'Laguna Woods' 'Coto De Caza' 'Laguna Hills' 'Brea' 'Foothill Ranch'
 'Midway City' 'Dana Point' 'Stanton' 'Lake Forest' 'Ladera Ranch'
 'La Habra Heights' 'La Palma' 'Laguna Beach' 'El Toro' 'Capistrano Beach'
 'Villa Park' 'Rossmoor' 'Trabuco Canyon' 'Fountain Vly' 'Portola Hills'
 'Rcho Sta Marg' 'Monarch Beach' 'Corona Del Mar' 'Dove Canyon' nan
 'Corona Dl Mar' 'Sunset Beach']
55


### Xem phần region

In [49]:
list_region = data_df.address_region.unique()
print(list_region)
print(len(list_region))

['CA' 'NV']
2


### Xem phần address code

In [51]:
list_address_code = data_df.address_code.unique()
print(list_address_code)
print(len(list_address_code))

[92603 92692 92646 90631 92705 92602 90620 92844 92694 92688 92708 92663
 92656 92833 92683 92807 92886 92676 92782 92870 92657 92618 92626 92604
 92647 92780 92673 90621 90720 92677 92801 92808 92612 92648 92620 90740
 92614 92701 92675 92835 92691 92649 92869 92637 92679 92704 92653 92843
 92802 92805 92823 92610 92655 92865 92629 92868 92703 92706 90680 92660
 92804 92831 92627 92707 92662 92840 92887 92866 92630 92624 92867 92672
 90623 90630 92651 92841 92661 92606 92821 92845 92806 92861 92832 92625
 92617 90742 92685]
87


### Xem phần type của căn nhà

In [52]:
list_type = data_df.info_type.unique()
print(list_type)
print(len(list_type))

['Condominium' 'Single Family Residence' 'Multi-Family Dwellings'
 'MISCELLANEOUS' 'Contact Agent' 'Cooperative'
 'Miscellaneous Structures - Ranch, Farm Fixtures'
 'Governmental / Public Use (general)' 'Commercial - Vacant Land'
 'Miscellaneous (Residential)' 'Duplex (2 units, any combination)'
 'Residential - Vacant Land' 'Agricultural (unimproved) - Vacant Land'
 'Miscellaneous (general)']
14


### Xem phần year built 

In [54]:
list_year_built = data_df.info_year_built.unique()
print(list_year_built)
print(len(list_year_built))

[1977. 1985.   nan 1980. 1964. 1965. 1955. 1972. 2000. 1970. 1975. 1989.
 1954. 1956. 1973. 1981. 1957. 1994. 1999. 1987. 1995. 1959. 2005. 1993.
 2006. 1978. 1958. 1988. 1974. 1960. 1983. 1967. 1950. 1969. 2004. 1976.
 1966. 1990. 1963. 1991. 1982. 1947. 2001. 1952. 1984. 2009. 2002. 1992.
 1998. 2014. 1986. 1962. 1979. 1946. 1996. 2012. 1940. 1914. 1923. 1948.
 1968. 1971. 1961. 1944. 1997. 2007. 1951. 2013. 1928. 1936. 1953. 1935.
 2003. 1949. 1926. 1930. 1907. 1916. 1938. 1933. 1942. 2011. 1929. 1921.
 1941. 1927. 2010. 1937. 2008. 1910. 1925. 1922. 1894. 1945. 1924. 1920.
 1880. 1913. 1919. 1939. 1909. 1917. 1912. 1904. 1915. 1932. 1931. 1908.
 1903. 1905. 1900. 1918. 1899. 1934. 1901. 1943. 1906. 1911.]
118


### Xử lý cột taxes_taxes

In [72]:
tmp = data_df.taxes_taxes.str.partition(' ')

In [75]:
# Xem phần tiền
tmp[0]

0         3947
1         4367
2        11464
3         4798
4         5210
5         8123
6        12770
7         5871
8         5673
9          NaN
10        6963
11        2649
12       17533
13        4029
14        2116
15        1092
16        2049
17        1886
18        4250
19        8457
20        8093
21        4971
22        6535
23        3399
24        7849
25       14553
26       19507
27        8898
28        1260
29        6576
         ...  
15981     3258
15982     6864
15983    18438
15984     6575
15985     1210
15986     1347
15987     3550
15988     9652
15989     2144
15990     3163
15991     5272
15992    39402
15993     1685
15994     2323
15995    10555
15996      930
15997    13293
15998     5073
15999     3572
16000     2322
16001     5830
16002     7995
16003    18206
16004     5076
16005     7922
16006     7056
16007     4881
16008     2127
16009    14757
16010    10555
Name: 0, Length: 16011, dtype: object

In [79]:
# Xem phần phần % thuế
tmp[2].str.replace("(","").str.replace(")","") # ảo 2 chỗ ".str" -> why?

0        1.07 %
1        1.01 %
2        1.12 %
3        1.01 %
4        1.34 %
5        1.12 %
6        1.58 %
7        1.17 %
8        1.26 %
9           NaN
10       1.08 %
11       1.21 %
12       1.08 %
13       0.98 %
14       1.24 %
15       2.22 %
16       1.22 %
17       1.32 %
18       1.29 %
19       1.06 %
20       1.13 %
21       1.12 %
22       1.21 %
23       1.04 %
24       1.15 %
25       1.10 %
26       1.57 %
27       1.12 %
28       1.13 %
29       1.15 %
          ...  
15981    1.02 %
15982    1.19 %
15983    1.65 %
15984    1.20 %
15985    1.45 %
15986    0.99 %
15987    1.14 %
15988    1.13 %
15989    0.98 %
15990    1.24 %
15991    1.13 %
15992    1.06 %
15993    0.99 %
15994    1.25 %
15995    1.12 %
15996    1.75 %
15997    1.46 %
15998    1.00 %
15999    1.01 %
16000    1.24 %
16001    1.16 %
16002    1.67 %
16003    1.07 %
16004    1.20 %
16005    1.25 %
16006    1.59 %
16007    1.18 %
16008    1.24 %
16009    1.09 %
16010    1.12 %
Name: 2, Length: 16011, 

### Xem thử xem các id căn nhà có độc lập không

In [81]:
len(data_df.info_property_id.unique()) # bằng 16011 thì ngon lành!

16011